In [39]:
import pandas as pd
from datasets import Dataset
data = pd.read_csv("./IMDB Dataset.csv")
texts = data.iloc[:,0].values
dict_list = [{"text1": text} for text in texts]
dataset = Dataset.from_list(dict_list)

In [40]:
from datasets import Dataset

dataset = Dataset.from_list(dict_list)
dataset[0]

{'text1': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due

In [41]:
import re

def f(data):
    text = []
    for i in data['text1']:
        i = i.lower().replace('<br />',' ')
        # 去除标点符号
        i = re.sub('\W', ' ', i) #\w 是非字母数字下划线
        # 去除连续空格

        i = re.sub('\s{2,}' , ' ',i)
        # 去除数字，避免记住集体数值
        i = re.sub( '\d+','<num>',i)
        #切词
        i = i.strip().split(' ')
        text.append(i)

    return{"text2" :text}


dataset = dataset.map(f,
                      batched=True,
                      batch_size=1000,
                      num_proc=4,
                      remove_columns=['text1'])


Map (num_proc=4): 100%|██████████| 50000/50000 [00:04<00:00, 12106.70 examples/s]


In [42]:
#过滤长度
def f_len(data):
    return [len(set(i)) > 12 + 6 + 1 for i in data['text2']]


dataset = dataset.filter(f_len, batched=True, batch_size=1000, num_proc=4)

len(dataset), dataset[0]

Filter (num_proc=4): 100%|██████████| 50000/50000 [00:04<00:00, 12033.53 examples/s]


(49963,
 {'text2': ['one',
   'of',
   'the',
   'other',
   'reviewers',
   'has',
   'mentioned',
   'that',
   'after',
   'watching',
   'just',
   '<num>',
   'oz',
   'episode',
   'you',
   'll',
   'be',
   'hooked',
   'they',
   'are',
   'right',
   'as',
   'this',
   'is',
   'exactly',
   'what',
   'happened',
   'with',
   'me',
   'the',
   'first',
   'thing',
   'that',
   'struck',
   'me',
   'about',
   'oz',
   'was',
   'its',
   'brutality',
   'and',
   'unflinching',
   'scenes',
   'of',
   'violence',
   'which',
   'set',
   'in',
   'right',
   'from',
   'the',
   'word',
   'go',
   'trust',
   'me',
   'this',
   'is',
   'not',
   'a',
   'show',
   'for',
   'the',
   'faint',
   'hearted',
   'or',
   'timid',
   'this',
   'show',
   'pulls',
   'no',
   'punches',
   'with',
   'regards',
   'to',
   'drugs',
   'sex',
   'or',
   'violence',
   'its',
   'is',
   'hardcore',
   'in',
   'the',
   'classic',
   'use',
   'of',
   'the',
   'word',

In [43]:
dataset[0]['text2']

['one',
 'of',
 'the',
 'other',
 'reviewers',
 'has',
 'mentioned',
 'that',
 'after',
 'watching',
 'just',
 '<num>',
 'oz',
 'episode',
 'you',
 'll',
 'be',
 'hooked',
 'they',
 'are',
 'right',
 'as',
 'this',
 'is',
 'exactly',
 'what',
 'happened',
 'with',
 'me',
 'the',
 'first',
 'thing',
 'that',
 'struck',
 'me',
 'about',
 'oz',
 'was',
 'its',
 'brutality',
 'and',
 'unflinching',
 'scenes',
 'of',
 'violence',
 'which',
 'set',
 'in',
 'right',
 'from',
 'the',
 'word',
 'go',
 'trust',
 'me',
 'this',
 'is',
 'not',
 'a',
 'show',
 'for',
 'the',
 'faint',
 'hearted',
 'or',
 'timid',
 'this',
 'show',
 'pulls',
 'no',
 'punches',
 'with',
 'regards',
 'to',
 'drugs',
 'sex',
 'or',
 'violence',
 'its',
 'is',
 'hardcore',
 'in',
 'the',
 'classic',
 'use',
 'of',
 'the',
 'word',
 'it',
 'is',
 'called',
 'oz',
 'as',
 'that',
 'is',
 'the',
 'nickname',
 'given',
 'to',
 'the',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'it',
 'focuses',
 'mainly',

In [44]:
import torch
    # 构建字典
def get_vocab():
    vocab = {}
    for i in range(len(dataset)):
        sent = dataset[i]['text2']
        for word in sent:
            if word not in vocab:
                vocab[word] = len(vocab)

        if i % 10000 == 0:
            print(i,len(vocab))
    
    return vocab
 
vocab = get_vocab()

len(vocab), vocab['girl']

0 189
10000 52104
20000 69094
30000 81402
40000 91684


(100507, 1206)

In [45]:
def f(data):
    sents = []
    for sent in data['text2']:
        sent = [vocab[word] for word in sent]
        sents.append(sent)

    return {"text":sents}

dataset = dataset.map(f,
                      batched=True,
                      batch_size=1000,
                      num_proc = 4)

Map (num_proc=4): 100%|██████████| 49963/49963 [00:06<00:00, 7465.87 examples/s] 


In [46]:
#转list
dataset = dataset['text']

len(dataset), dataset[0]

(49963,
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  2,
  29,
  30,
  7,
  31,
  28,
  32,
  12,
  33,
  34,
  35,
  36,
  37,
  38,
  1,
  39,
  40,
  41,
  42,
  20,
  43,
  2,
  44,
  45,
  46,
  28,
  22,
  23,
  47,
  48,
  49,
  50,
  2,
  51,
  52,
  53,
  54,
  22,
  49,
  55,
  56,
  57,
  27,
  58,
  59,
  60,
  61,
  53,
  39,
  34,
  23,
  62,
  42,
  2,
  63,
  64,
  1,
  2,
  44,
  65,
  23,
  66,
  12,
  21,
  7,
  23,
  2,
  67,
  68,
  59,
  2,
  69,
  70,
  71,
  72,
  73,
  65,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  1,
  2,
  82,
  83,
  84,
  2,
  85,
  86,
  87,
  88,
  36,
  89,
  90,
  91,
  92,
  23,
  47,
  93,
  76,
  2,
  94,
  95,
  78,
  23,
  96,
  59,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  36,
  105,
  91,
  106,
  107,
  108,
  109,
  110,
  36,
  111,
  112,
  19,
  113,
  114,
  115,
  116,
  117,
  118,
  

##使用传统数据导入

In [47]:
import random


def get_data():
    text = random.choice(dataset)

    i = random.randint(3, len(text) - 4)
    x = text[i]

    pos = text[i - 3:i + 4]
    pos.pop(3)

    neg = []
    while len(neg) != 12:
        neg = random.sample(text, 12 + 6)
        neg = [i for i in neg if i not in pos]
        neg = neg[:12]

    return x, pos, neg


get_data()

(985,
 [474, 50, 0, 7, 23, 10597],
 [826, 11, 59, 4224, 209, 247, 119, 1, 36, 139, 173, 11])

In [48]:
import torch


def get_batch(batch_size = 128):
    x = []
    pos = []
    neg = []
    for _ in range(batch_size):
        data = get_data()
        x.append(data[0])
        pos.append(data[1])
        neg.append(data[2])

    x = torch.LongTensor(x)
    pos = torch.LongTensor(pos)
    neg = torch.LongTensor(neg)

    return x, pos, neg


get_batch()

(tensor([ 5643,  1273,     2,    14, 15579,    22,    33,    36,  1266,    23,
           164,  3426,   518, 29204,  1904,  6248,     2,  2356,    65,   116,
          1545,  1322,  4745, 10667,  1434,  5321,   173,  2415,   669,    36,
            86,  1281,  1054,    18,   481,  9117,  2093,    50,  4154,  1599,
            23,  2365,     2,    50,     1,    10,  2157,     2,   726,     2,
           116, 24900,     2,   324,   296,    40,   116,    36,   184,  7548,
            43,  1751,   909,   432,    22,   514,   686,  1137, 89593,  3050,
             1,   105,   247,    65,   156, 19397,    59,   240,  1277,  4844,
            36,   270,    59,   126,  3076,  1671,  8648,   145,   294,   230,
            59,   827,   355,    32,    11,  6805,  2407,     0,    59,   648,
           989,   662,  6358,     2,    19,    22,   485,   320, 49978,    65,
          1294,   416,  1519,     2,  5455,   116,   443, 19185,    36,    23,
          1657,     2,  1457,     1,  2547,     2,  

In [49]:
def get_cos_loss(x, pos, neg):
    #x -> [8, 150]
    #pos -> [8, 6, 150]
    #neg -> [8, 12, 150]

    #[8, 150] -> [8, 150, 1]
    x = x.unsqueeze(dim=2)

    #[8, 6, 150],[8, 150, 1] -> [8, 6]
    loss_pos = torch.bmm(pos, x).squeeze(dim=2)

    #[8, 12, 150],[8, 150, 1] -> [8, 12]
    loss_neg = torch.bmm(neg, -x).squeeze(dim=2)

    #[8, 6] -> [8]
    loss_pos = loss_pos.sigmoid().clip(min=1e-8).log().sum(dim=1)

    #[8, 12] -> [8]
    loss_neg = loss_neg.sigmoid().clip(min=1e-8).log().sum(dim=1)

    return -(loss_pos + loss_neg).mean()


a, b, c = torch.randn(8, 150), torch.randn(8, 6, 150), torch.randn(8, 12, 150)
get_cos_loss(a, b, c)

tensor(80.3872)

In [50]:
import torch.utils


class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.embed = torch.nn.Embedding(num_embeddings=len(vocab),
                                        embedding_dim=150)
        
        self.embed.weight.data.uniform_(-0.01,0.01)

    def forward(self, x, pos, neg):
        #编码
        #[8] -> [8, 2]
        x = self.embed(x)

        #[8, 6] -> [8, 6, 2]
        pos = self.embed(pos)

        #[8, 12] -> [8, 12, 2]
        neg = self.embed(neg)

        return get_cos_loss(x, pos, neg)
    


In [51]:
model = Model()
model(*get_batch())## test

tensor(12.4771, grad_fn=<NegBackward0>)

In [52]:
def test(test_words):
    embed = model.embed.weight.data.clone()
    vocab_keys = list(vocab.keys())

    for word in test_words:
        x = embed[vocab[word]]
        score = torch.nn.functional.cosine_similarity(x, embed)
        topk = score.topk(k=5).indices
        topk = [vocab_keys[k] for k in topk]
        print(word, topk)


test(['girl', 'bus', 'green', 'doctor', 'dog', 'queen', 'italy'])

girl ['girl', 'guided', 'ney', 'referent', 'erect']
bus ['bus', 'groovebox', 'defensible', 'dikker', 'hotmail']
green ['green', 'rereads', 'nude', 'knit', 'gooble']
doctor ['doctor', 'rifled', 'revitalize', 'nerdiness', 'sugary']
dog ['dog', 'dimbulb', 'tarazu', 'ringer', 'deez']
queen ['queen', 'dunneare', 'gruenberg', 'tdc', 'cegid']
italy ['italy', 'arahan', 'musics', 'abetting', 'visage']


In [53]:
from tqdm import tqdm
import os
def train():
    global model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    print(device)
    os.makedirs('models', exist_ok=True) 
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    loss_sum = 0
    for epoch in tqdm(range(200001)):
        batch = get_batch(batch_size=128)
        batch = [i.to(device) for i in batch]

        loss = model(*batch)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        loss_sum += loss.item()

        if epoch % 100000 == 0:
            print(epoch, loss_sum/10000)
            test(['girl', 'bus', 'green', 'doctor', 'dog', 'queen', 'italy'])
            loss_sum = 0
        if epoch % 100000 == 0:
                    torch.save(model.cpu(), 'models/imdb_%d.model' % epoch)
                    model = model.to(device)

    model = model.cpu()

In [54]:
train()

cuda


  0%|          | 1/200001 [00:00<6:24:20,  8.67it/s]

0 0.001247706413269043
girl ['girl', 'guided', 'toooooo', 'ney', 'curb']
bus ['bus', 'groovebox', 'defensible', 'dikker', 'hotmail']
green ['green', 'rereads', 'nude', 'knit', 'gooble']
doctor ['doctor', 'rifled', 'sugary', 'impalpable', 'weybridge']
dog ['dog', 'dimbulb', 'tarazu', 'deez', 'tuareg']
queen ['queen', 'dunneare', 'gruenberg', 'tdc', 'cegid']
italy ['italy', 'arahan', 'musics', 'abetting', 'visage']


 50%|█████     | 100005/200001 [25:55<30:26, 54.75it/s]

100000 123.33982315912246
girl ['girl', 'boy', 'woman', 'young', 'man']
bus ['bus', 'whispering', 'snell', 'munched', 'gurly']
green ['green', 'gables', 'soylent', 'gleen', 'architecture']
doctor ['doctor', 'insinnia', 'nun', 'who', 'reporter']
dog ['dog', 'lazaro', 'scythe', 'pizza', 'kaas']
queen ['queen', 'snow', 'shiksa', 'masssacre', 'manon']
italy ['italy', 'england', 'japan', 'in', 'australia']


100%|██████████| 200001/200001 [51:50<00:00, 64.31it/s]

200000 123.27152564458846
girl ['girl', 'boy', 'young', 'woman', 'man']
bus ['bus', 'fremont', 'duffle', 'peaknuckle', 'cout']
green ['green', 'cloudless', 'blue', 'red', 'gables']
doctor ['doctor', 'kildaire', 'girl', 'suess', 'jewish']
dog ['dog', 'lazaro', 'puppy', 'horse', 'hoarder']
queen ['queen', 'clytemnastrae', 'victoria', 'hathcocks', 'bee']
italy ['italy', 'excursionists', 'gore_won', 'france', 'southeastern']


###使用Dataset+Dataloder

In [55]:
import random
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, dataset):
        """
        Args:
            dataset (List[List]): 一个由句子组成的列表，每个元素是一个词列表
        """
        self.dataset = dataset
        self.samples = []
        for text in dataset:
            if len(text) >= 7:
                for i in range(3, len(text) - 3):
                    self.samples.append((text, i))

    def __len__(self):
        return len(self.dataset)  # 可以理解为有多少个样本（句子）

    def __getitem__(self, idx):
        # idx 是 DataLoader 自动传入的索引，但我们随机选一个句子
        text, i = self.samples[idx]  # ✅ 无需 runtime 再随机
        x = text[i]
        pos = text[i - 3:i + 4]
        pos.pop(3)

        neg = []
        while len(neg) < 12:
            candidates = random.sample(text, min(12 + 6, len(text)))
            neg = [w for w in candidates if w not in pos]
        neg = neg[:12]
        

        # x = torch.LongTensor([x])
        pos = torch.LongTensor(pos)
        neg = torch.LongTensor(neg)

        return x, pos, neg

In [56]:
from torch.utils.data import DataLoader


my_dataset = MyDataset(dataset)
loader = DataLoader(
    my_dataset,
    batch_size=128,
    shuffle=False,  # 因为我们内部已经随机选句子了
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
    # collate_fn=lambda batch: batch  # 不做 stack，返回 list of tuples
)

In [57]:
for batch in loader:
    x, pos, neg = batch
    # x.squeeze(dim=1)
    break
    pass
x.shape, pos.shape, neg.shape

(torch.Size([128]), torch.Size([128, 6]), torch.Size([128, 12]))

In [58]:
from torch.utils.data import random_split
import random
from torch.utils.data import Subset

class EarlyStopping:
    def __init__(self, patience=5, delta=0.0):
        self.patience = patience      # 容忍次数
        self.delta = delta            # 最小提升值
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            print(f"EarlyStopping: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def validate():
  
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    indices = random.sample(range(len(my_dataset)), 200)  # 随机取200个索引
    val_dataset = Subset(my_dataset, indices=indices)
    val_loader = DataLoader(val_dataset, batch_size=64)
    # val_loader = DataLoader(my_dataset, batch_size=64)
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            x, pos, neg = batch
            x = x.to(device)
            pos = pos.to(device)
            neg = neg.to(device)
            loss = model(x, pos, neg)
            total_loss += loss.item()
    return total_loss / len(val_loader)


In [59]:
from tqdm import tqdm
def train():
    global model
    early_stopper = EarlyStopping(patience=100, delta=0.001)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    print(device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    loss_sum = 0
    t = 0
    for epoch in tqdm(range(2001)):

        for batch in loader:
            x, pos, neg = batch  # unpack 一个 batch 中的三个部分

            x = x.to(device)     # shape: (1,)
            pos = pos.to(device) # shape: (1, 6)
            neg = neg.to(device) # shape: (1, 12)

            loss = model(x, pos, neg)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            # print(loss)
            loss_sum += loss.item()
        t+=1    
        # print(f"{t}次迭代")
        if epoch % 1000 == 0:
            val_loss = validate()
            
            print(f"Epoch {epoch} | Train Loss: {loss_sum/1000:.4f} | Val Loss: {val_loss:.4f}")
            early_stopper(val_loss)
            loss_sum = 0
        if epoch % 1000 == 0:
            print(epoch, loss_sum/10)
            test(['girl', 'bus', 'green', 'doctor', 'dog', 'queen', 'italy'])
            # loss_sum = 0



        if epoch % 100000 == 0:
            # torch.save(model.cpu(), 'models_dataloder/imdb_%d.model' % epoch)
            # model = model.to(device)
            pass

    model = model.cpu()

train()

cuda


  0%|          | 1/2001 [00:00<28:45,  1.16it/s]

Epoch 0 | Train Loss: 4.8217 | Val Loss: 12.0714
0 0.0
girl ['girl', 'boy', 'young', 'woman', 'man']
bus ['bus', 'fremont', 'snouts', 'peaknuckle', 'duffle']
green ['green', 'cloudless', 'red', 'blue', 'gables']
doctor ['doctor', 'girl', 'kildaire', 'suess', 'loving']
dog ['dog', 'lazaro', 'puppy', 'geurilla', 'hoarder']
queen ['queen', 'clytemnastrae', 'victoria', 'brobdingnag', 'hathcocks']
italy ['italy', 'excursionists', 'gore_won', 'france', 'helsinki']


 50%|█████     | 1001/2001 [13:53<14:04,  1.18it/s]

Epoch 1000 | Train Loss: 4151.9016 | Val Loss: 10.4414
1000 0.0
girl ['girl', 'crazed', 'standing', 'corpses', 'blind']
bus ['bus', 'down', 'getting', 'way', 'adjournment']
green ['green', 'card', 'gasoline', 'puchase', 'naaahhh']
doctor ['doctor', 'mixed', 'breed', 'reporter', 'substandard']
dog ['dog', 'puppy', 'fetishists', 'boardinghouse', 'chist']
queen ['queen', 'clown', 'damned', 'moustache', 'viciousness']
italy ['italy', 'england', 'salerno', 'stros', 'produce']


100%|██████████| 2001/2001 [27:49<00:00,  1.20it/s]

Epoch 2000 | Train Loss: 3960.6573 | Val Loss: 10.0834
2000 0.0
girl ['girl', 'plummets', 'standing', 'crazed', 'cute']
bus ['bus', 'getting', 'down', 'intercontinental', 'tells']
green ['green', 'card', 'gasoline', 'puchase', 'naaahhh']
doctor ['doctor', 'mixed', 'breed', 'reporter', 'crumby']
dog ['dog', 'fetishists', 'horn', 'puppy', 'eyed']
queen ['queen', 'clown', 'moustache', 'damned', 'drag']
italy ['italy', 'salerno', 'england', 'southern', 'palermo']


In [60]:
model = torch.load('models/imdb_200000.model')

test(['girl', 'bus', 'green', 'doctor', 'dog', 'queen', 'italy'])

girl ['girl', 'boy', 'young', 'woman', 'man']
bus ['bus', 'fremont', 'duffle', 'peaknuckle', 'cout']
green ['green', 'cloudless', 'blue', 'red', 'gables']
doctor ['doctor', 'kildaire', 'girl', 'suess', 'jewish']
dog ['dog', 'lazaro', 'puppy', 'horse', 'hoarder']
queen ['queen', 'clytemnastrae', 'victoria', 'hathcocks', 'bee']
italy ['italy', 'excursionists', 'gore_won', 'france', 'southeastern']
